In [3]:
import boto3

REGION = boto3.Session().region_name

sagemaker_client = boto3.client("sagemaker", region_name=REGION)
pipeline_name="neptune-ml-sagemaker-pipeline-movie-test-sm-da-1"

# Replace with your S3 bucket where you prepared the IMDB data for Neptune use
SOURCE_S3_PATH_NOSLASH=f"s3://ekg-da-poc"

# Replace with your Amazon OpenSearch Service domain endpoint
FTS_AOS_ENDPOINT="https://z9urozgne3v5p4ywe2fj.us-east-1.aoss.amazonaws.com"

VS_AOS_ENDPOINT="https://8kd01utb7q4h0s0lec3g.us-east-1.aoss.amazonaws.com"

# Replace with your IAM role for the GNN embedding pipeline.
SAGEMAKER_ROLE="arn:aws:iam::196856463470:role/SandboxServiceRole"

sagemaker_client.create_pipeline(
    PipelineName=pipeline_name,
    PipelineDefinitionS3Location={
        'Bucket': f"graphlytics-{REGION}",
        'ObjectKey': "v1/sagemaker-pipelines/neptune-ml-training-pipeline.json",
        },
   RoleArn=SAGEMAKER_ROLE
)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:196856463470:pipeline/neptune-ml-sagemaker-pipeline-movie-test-sm-da-1',
 'ResponseMetadata': {'RequestId': '1ee5e490-1954-4067-8a0a-580cd7b86021',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1ee5e490-1954-4067-8a0a-580cd7b86021',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Sat, 31 Aug 2024 21:17:46 GMT'},
  'RetryAttempts': 0}}

In [4]:
input_data_s3_location=f"{SOURCE_S3_PATH_NOSLASH}/source"
processed_data_s3_location = f"{SOURCE_S3_PATH_NOSLASH}/processing"
train_model_s3_location = f"{SOURCE_S3_PATH_NOSLASH}/training"
embedding_s3_location = f"{SOURCE_S3_PATH_NOSLASH}/embeddings"
processing_instance_type = "ml.r5.4xlarge"
training_instance_type="ml.p3.8xlarge"
processing_instance_size_gb="200"
training_instance_size_gb="200"

start_result = sagemaker_client.start_pipeline_execution(
    PipelineName=pipeline_name,
    PipelineParameters=[

    { "Name": "inputDataS3Location", "Value": input_data_s3_location },
    { "Name": "processedDataS3Location", "Value": processed_data_s3_location},
    { "Name": "trainModelS3Location", "Value": train_model_s3_location},
    { "Name": "embeddingS3Location", "Value": embedding_s3_location},
    { "Name": "embeddingDimension", "Value": "64"},
    { "Name": "model", "Value": "rgcn"},
    { "Name": "sagemakerIamRoleArn", "Value": SAGEMAKER_ROLE},
    { "Name": "processingInstanceType", "Value": processing_instance_type},
    { "Name": "trainingInstanceType", "Value": training_instance_type},
    { "Name": "processingInstanceVolumeSizeInGB", "Value": processing_instance_size_gb},
    { "Name": "trainingInstanceVolumeSizeInGB", "Value": training_instance_size_gb}
])

print(start_result)

{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:196856463470:pipeline/neptune-ml-sagemaker-pipeline-movie-test-sm-da-1/execution/skznly4oqoh6', 'ResponseMetadata': {'RequestId': 'faef4a65-322d-4503-b653-835da66eae34', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'faef4a65-322d-4503-b653-835da66eae34', 'content-type': 'application/x-amz-json-1.1', 'content-length': '148', 'date': 'Sat, 31 Aug 2024 21:18:30 GMT'}, 'RetryAttempts': 0}}


In [5]:
arn=start_result['PipelineExecutionArn']
print(arn)

sagemaker_client.describe_pipeline_execution(
    PipelineExecutionArn=arn
)

arn:aws:sagemaker:us-east-1:196856463470:pipeline/neptune-ml-sagemaker-pipeline-movie-test-sm-da-1/execution/skznly4oqoh6


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:196856463470:pipeline/neptune-ml-sagemaker-pipeline-movie-test-sm-da-1',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:196856463470:pipeline/neptune-ml-sagemaker-pipeline-movie-test-sm-da-1/execution/skznly4oqoh6',
 'PipelineExecutionDisplayName': 'execution-1725139111560',
 'PipelineExecutionStatus': 'Failed',
 'PipelineExperimentConfig': {'ExperimentName': 'neptune-ml-sagemaker-pipeline-movie-test-sm-da-1',
  'TrialName': 'skznly4oqoh6'},
 'FailureReason': 'Step failure: One or multiple steps failed.',
 'CreationTime': datetime.datetime(2024, 8, 31, 21, 18, 31, 458000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 31, 21, 18, 34, 924000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:196856463470:user-profile/d-yy2x9kkg6qqh/pd-1718822752509',
  'UserProfileName': 'pd-1718822752509',
  'DomainId': 'd-yy2x9kkg6qqh',
  'IamIdentity': {'Arn': 'arn:aws:sts::196856463470:assumed-role